## Always run the first three cells below  

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import dihedrals, rms
from MDAnalysis.lib.distances import calc_dihedrals 

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import numpy as np
import nglview as nv
import pandas as pd
import math
import sys

### Set parameters 

In [ ]:
is_deactivate_warning = True
is_savefigs = True

sysname = 'AlanineDipeptide'
# name of PDB file
pdb_filename = "./vacuum.pdb"
# name of DCD file
output_path = './traj_data' 

traj_dcd_filename = '%s/traj.dcd' % output_path

if is_deactivate_warning :
    import warnings
    warnings.filterwarnings("ignore")    

### Load trajectory data 

In [ ]:
# load the trajectory data from DCD file
u = mda.Universe(pdb_filename, traj_dcd_filename)
# load the reference configuration from the PDB file
ref = mda.Universe(pdb_filename) 

# print some information
print ('residues: ', u.residues)
print ('trajectory: ', u.trajectory)
print ('reference: ', ref.trajectory)

# display the trajectory
view = nv.show_mdanalysis(u)
print ('number of frames: %d ' % view.max_frame)
view

## Select from Jobs 1-5, or  run them sequentially. 

### Job 1: Generate plots of two dihedral angles

In [ ]:
# generate the Ramachandran plot of two dihedral angles
#ax = plt.gca()
fig, ax = plt.subplots(figsize=(7,7))

#phi = u.select_atoms("bynum 13 15 17 1")
#psi = u.select_atoms("bynum 13 15 17 1")
#r = dihedrals.Dihedral([phi,psi]).run()

traj = u.trajectory.timeseries(order='fac')

phi_angles = calc_dihedrals(traj[:,12,:], traj[:,14,:], traj[:,16,:], traj[:,1,:]) / math.pi * 180
psi_angles = calc_dihedrals(traj[:,2,:], traj[:,0,:], traj[:,16,:], traj[:,14,:]) / math.pi * 180

h = ax.hist2d(phi_angles, psi_angles, bins=70, norm=mpl.colors.LogNorm(), range=[[-180,180], [-180, 180]]) #norm=mpl.colors.LogNorm(),

#r = dihedrals.Ramachandran(u.select_atoms('resid 2')).run()
#r.plot(ax, color='black', marker='.') #, ref=True)

traj_angles = np.stack((phi_angles, psi_angles), axis=1)

if is_savefigs :
    fig_filename_Ramachandran = './%s/%s_Ramachandran.eps' % (output_path, sysname)
    plt.savefig(fig_filename_Ramachandran)
    print ('Ramachandran plot saved to file: %s' % fig_filename_Ramachandran)
    
fig, ax = plt.subplots(1,2, figsize=(14,6))
ax[0].plot(traj_angles[:,0], 'k-')
ax[0].set_ylim([-180,180])
ax[0].set_xlabel("time (fs)")
ax[0].set_ylabel(r"$\phi$")

ax[1].plot(traj_angles[:,1], 'k-')
ax[1].set_xlabel("time (fs)")
ax[1].set_ylim([-180,180])
ax[1].set_ylabel(r"$\psi$")

plt.show()

### Job 2: Plot weights

In [ ]:
weight_filename = "%s/weights.csv" % output_path
weights = pd.read_csv(weight_filename, header=None)
print (weights.columns)
print ('\nDescribe:\n', weights.describe(percentiles=[0.2, 0.4, 0.6, 0.8]))

fig, ax = plt.subplots(1,2, figsize=(14,7))

nbin=10
[vmin, vmax] = [1e-8, 2e1]
[min_weight, max_weight] = [1e-8, 2e1]

# select states according to weights
select = (weights[0]>min_weight) & (weights[0]<max_weight)

weights = weights[select]
print ('\nDescribe:\n', weights.describe(percentiles=[0.2, 0.4, 0.6, 0.8]))

# histogram of weights
weights.plot(kind='hist', logx=True, logy=True, ax=ax[0], bins=nbin)

try:
    traj_angles 
except NameError:   
    r = dihedrals.Ramachandran(u.select_atoms('resid 2')).run()
    traj_angles = r.angles[:,0,:]

# scatter plot of weights 
sc = ax[1].scatter(traj_angles[select,0], traj_angles[select,1], s=3.0, \
                   c=weights, vmin=vmin, vmax=vmax, cmap='jet', norm=colors.LogNorm())

ax[1].set_title(f'weights', fontsize=27)
ax[1].set_xlim([-180,180])
ax[1].set_ylim([-180,180])

#ax.set_xticks([-3, -2, -1, 0, 1, 2, 3])
#ax.set_yticks([-3, -2, -1, 0, 1, 2, 3])
cax = fig.add_axes([0.92, 0.10, .02, 0.80])
cbar = fig.colorbar(sc, cax=cax)
cbar.ax.tick_params(labelsize=20)

plt.show()

### Job 3: RMSD plot

In [ ]:
# select atoms for RMSD computation below 
selector = 'name N or name CA or name C'
# compute RMSD of the trajectory wrt the reference configuration
R = rms.RMSD(u, ref, select=selector)          
R.run()
# get the RMSD result
rmsd = R.results.rmsd.T   # transpose makes it easier for plotting

fig, ax = plt.subplots(figsize=(12,6))

plt.plot(rmsd[0,:], rmsd[2,:], 'k-')
plt.xlabel("time (fs)")
plt.ylabel(r"RMSD ($\AA$)")

if is_savefigs :
    fig_filename_rmsd = './%s/%s_rmsd.eps' % (output_path, sysname) 
    plt.savefig(fig_filename_rmsd)    
    print ('RMSD plot saved to file: %s' % fig_filename_rmsd)

### Job 4: Plot state data from CSV file

In [ ]:
csv_filename = "%s/state_data.csv" % output_path
csv_col_idx = 1

df1 = pd.read_csv(csv_filename)
print (df1.describe())

fig, ax = plt.subplots(figsize=(8,6))
df1.plot(kind='line', ax=ax, x='#"Time (ps)"', y=df1.columns[csv_col_idx])

if is_savefigs :
    # to get rid of warning when saving figure to eps format
    ax.set_rasterized(True)
    fig_filename= './%s/%s_csv%d.eps' % (output_path, sysname, csv_col_idx) 
    plt.savefig(fig_filename)    
    print ('Potential Energy saved to file: %s' % fig_filename)

try:
    traj_angles 
except NameError:   
    r = dihedrals.Ramachandran(u.select_atoms('resid 2')).run()
    traj_angles = r.angles[:,0,:]
    
fig, ax = plt.subplots(figsize=(8,6))
sc = ax.scatter(traj_angles[:,0], traj_angles[:,1], s=3.0, c=df1[df1.columns[csv_col_idx]], cmap='jet')

ax.set_title(f'{df1.columns[csv_col_idx]}', fontsize=27)
#ax.set_xticks([-3, -2, -1, 0, 1, 2, 3])
#ax.set_yticks([-3, -2, -1, 0, 1, 2, 3])

cax = fig.add_axes([0.92, 0.10, .02, 0.80])
cbar = fig.colorbar(sc, cax=cax)
cbar.ax.tick_params(labelsize=20)
plt.show()

### Job 5: Check trained model on data

In [ ]:
from molann.feature import FeatureFileReader
import molann.ann as ann
import torch

num_scatter_states = 10000
feature_file = './feature.txt'
# read configuration parameters

#model_file = 'checkpoint/AlanineDipeptide-eigenfunction-2022-04-14-11:12:35/trained_cv_scripted.pt'
model_file = 'checkpoint/AlanineDipeptide-Eigenfunction-2022-05-18-14:43:37/latest/scripted_cv_cpu.pt'

print ('====================Trajectory Info===================')

# load the trajectory data from DCD file
traj = u.trajectory.timeseries(order='fac')

print ('==============Features===================\n')
print ('Features file: {}'.format(feature_file)) 

print ('\nFeatures for histogram: \n')
# read features for histogram plot
feature_reader = FeatureFileReader(feature_file, 'Histogram', u)
feature_list = feature_reader.read()


input_selector = 'type C or type O or type N'
input_ag = u.select_atoms(input_selector)
input_atom_indices = input_ag.ids - 1
traj = traj[:, input_atom_indices, :]

if len(feature_list) > 0 : 
    histogram_feature_mapper = ann.FeatureLayer(feature_list, input_ag, use_angle_value=True)
    print (histogram_feature_mapper.get_feature_info())
    # make sure each feature is one-dimensional
    assert histogram_feature_mapper.output_dimension() == len(feature_list), "Features for histogram are incorrect, output of each feature must be one-dimensional!" 

    histogram_feature = histogram_feature_mapper(torch.tensor(traj)).detach().numpy()
    feature_names = histogram_feature_mapper.get_feature_info()['name']
    df = pd.DataFrame(data=histogram_feature, columns=feature_names) 

    df.hist(figsize=(7,7))
    
    df.plot(figsize=(9,6), subplots=True)
    plt.legend(loc='best')

print ('\nFeatures for ouptut: \n')
# features to define a 2d space for output
feature_reader = FeatureFileReader(feature_file, 'Output', u) 
feature_list= feature_reader.read()
if len(feature_list) == 2 :
    output_feature_mapper = ann.FeatureLayer(feature_list, input_ag, use_angle_value=True)
    if output_feature_mapper.output_dimension() == 2 : # use it only if it is 2D
        print (output_feature_mapper.get_feature_info())
    else :
        print (f'\nOutput feature mapper set to None, since 2d feature required for output.')
        output_feature_mapper = None
else :
    output_feature_mapper = None

if output_feature_mapper :
    model = torch.jit.load(model_file)
    
    output_features = output_feature_mapper(torch.tensor(traj)).detach().numpy()

    index = np.random.choice(np.arange(traj.shape[0], dtype=int), num_scatter_states, replace=False)

    X = torch.tensor(traj)[index,:]
    feature_data = output_features[index,:]
    cv_vals = model(X) 

    k = cv_vals.size(1)

    for idx in range(k) :
        fig, ax = plt.subplots(figsize=(6,5))
        sc = ax.scatter(feature_data[:,0], feature_data[:,1], s=2.0, c=cv_vals[:,idx].detach().numpy(), cmap='jet')

        ax.set_title(f'{idx+1}th CV', fontsize=27)
        ax.set_xlabel(r'{}'.format(output_feature_mapper.get_feature(0).get_name()), fontsize=25, labelpad=3, rotation=0)
        ax.set_xticks([-3, -2, -1, 0, 1, 2, 3])
        ax.set_yticks([-3, -2, -1, 0, 1, 2, 3])
        ax.set_ylabel(r'{}'.format(output_feature_mapper.get_feature(1).get_name()), fontsize=25, labelpad=-10, rotation=0)

        cax = fig.add_axes([0.92, 0.10, .02, 0.80])
        cbar = fig.colorbar(sc, cax=cax)
        cbar.ax.tick_params(labelsize=20)
